In [1]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
from selenium.webdriver.common.by import By

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration (optional, but recommended for headless mode)
chrome_options.add_argument("--window-size=1920x1080")  # Optional: set window size for better rendering


# get set parts

In [3]:
def get_set_parts(url):
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,"html.parser")

    parts = []
    for line in soup.find_all("a"):
        test = str(line).split("Part")
        if len(test)>1: 
            test2 =(str(test[1]).split(" ")) 
            if len(test2) >4 and test2[1]=="No:":
                  parts.append(test2[2])

    qnt = []
    for line in soup.find_all("td", align="RIGHT"):
            test = re.findall(r'(\d+)', str(line))  # Find digits within the td tag
            if len(test)==1:
                qnt.append(test)

    return {(parts[i]):int(qnt[i][0]) for i in range(len(parts))}


# buy

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re


In [38]:
def search_for_part(part, driver):
    # Wait for the search box to be available
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "q"))
    )
    search_box.clear()
    search_box.send_keys(part)
    search_box.send_keys(Keys.RETURN)

    # Parse the page source
    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:
        # Wait for the dropdown menu for sorting to be available
        sort_select_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.NAME, "sort"))
        )
        # Create a Select object and choose the "Price" option
        sort_select = Select(sort_select_element)
        sort_select.select_by_value("6")  # "6" corresponds to "Price"
        return 1  # Found parts!

    except:
        return 0

    finally:
        # Ensure implicit wait is set regardless of the outcome
        driver.implicitly_wait(1)


def get_store_parts(driver):

    WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "buy"))
        )

    # get soup after page loads
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # get qnt:price dict
    items = soup.find_all("div", class_="buy")   
    price_dict = {}
    for item in items:
        a = (item.text.replace(",",""))

        numbers = re.findall(r'\d+\.\d+|\d+', a)
        if len(numbers) >= 2:
            price_dict[re.findall(r'\d+\.\d+|\d+', a.split("EUR")[0])[0]] = re.findall(r'\d+\.\d+|\d+', a.split("EUR")[1])[0] 
    return price_dict

def get_part_price(part_number, part_qnt, store_part_dict):

    for available_part_qnt in store_part_dict.keys():
            if int(available_part_qnt)>=part_qnt:
                    price = part_qnt*float(store_part_dict[available_part_qnt])
                    break
            else:
                    price = None
    return  price

In [6]:
def analyse(parts_dict, final_dict):

    parts_found = [part for part in final_dict.keys() if final_dict[part] is not None]
    price_found = sum([final_dict[part] for part in final_dict.keys() if final_dict[part] is not None])

    print(f"Found {len(parts_found)} out of {len(parts_dict.keys())} parts, around {100*len(parts_found)/len(parts_dict.keys()):.2f} %")
    print(f"These parts go by {price_found:.2f} euros!")

        
    not_found = [part for part in final_dict.keys() if final_dict[part] is  None]
    remaining_dict = {part:parts_dict[part] for part in not_found}

    return remaining_dict

In [7]:
stores_url = "https://www.bricklink.com/browse.asp"
def get_stores(stores_url,nmr_of_stores = 5):

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(stores_url)
    driver.implicitly_wait(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    store_list = soup.find_all("table", class_ = "store-list")
    european_store_list = [store.find_all("a", href = True) for store in store_list if store.find("th", string = "Europe") is not None and store.find("tr", class_ = "divider") is None]

    urls = []
    names = []
    for store_list in european_store_list:
        for store in store_list:
            
            store_ = str(store).split('"')
            if len(store_)==3:
                urls.append(store_[1])
                names.append(store_[2])

    full_dict = {names[i]:urls[i] for i in range(len(urls))}
    return {store:full_dict[store] for store in list(full_dict.keys())[0:nmr_of_stores]}



In [8]:
def main(store_url,set_parts_dict):
    driver = webdriver.Chrome()
    driver.get(store_url)
    final_dict = {}

    for part_number, part_qnt in set_parts_dict.items():
        # search
        if search_for_part(part_number,driver):

            # get parts
            store_part_dict = get_store_parts(driver)
            
            final_dict[part_number] = get_part_price(part_number, part_qnt, store_part_dict)
        else:
            final_dict[part_number] = None
    
        
    return final_dict

In [9]:
magic_magnus_store_url = "https://store.bricklink.com/MagicMagnus?p=MagicMagnus#/shop?o={%22q%22:%2232014%22,%22sort%22:6,%22showHomeItems%22:0}"


volvo_set_url = "https://www.bricklink.com/catalogItemInv.asp?S=42053-1"

In [9]:
volvo_parts = get_set_parts(volvo_set_url)
magic_magnus_parts = main(magic_magnus_store_url, volvo_parts)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00007FF7A2976CB5+28821]
	(No symbol) [0x00007FF7A28E3840]
	(No symbol) [0x00007FF7A278578A]
	(No symbol) [0x00007FF7A275F4F5]
	(No symbol) [0x00007FF7A2806247]
	(No symbol) [0x00007FF7A281ECE2]
	(No symbol) [0x00007FF7A27FF0A3]
	(No symbol) [0x00007FF7A27CA778]
	(No symbol) [0x00007FF7A27CB8E1]
	GetHandleVerifier [0x00007FF7A2CAFCAD+3408013]
	GetHandleVerifier [0x00007FF7A2CC741F+3504127]
	GetHandleVerifier [0x00007FF7A2CBB5FD+3455453]
	GetHandleVerifier [0x00007FF7A2A3BDBB+835995]
	(No symbol) [0x00007FF7A28EEB5F]
	(No symbol) [0x00007FF7A28EA814]
	(No symbol) [0x00007FF7A28EA9AD]
	(No symbol) [0x00007FF7A28DA199]
	BaseThreadInitThunk [0x00007FFC26FE259D+29]
	RtlUserThreadStart [0x00007FFC28C8AF38+40]


In [ ]:
remaining_parts = analyse(volvo_parts, magic_magnus_parts)

Found 105 out of 152 parts, around 69.08 %
These parts go by 29.58 euros!


In [10]:
store_pre_http = "https://store.bricklink.com/"

In [681]:
all_stores_dict = get_stores(stores_url)



In [682]:
list(all_stores_dict.values())[0]

'/store.asp?p=tomte'

In [684]:
evr = []
for store in all_stores_dict.keys():
    store_parts = main(store_pre_http+all_stores_dict[store],remaining_parts )
    evr.append(store_parts)
    print("store:", store)
    this_store_remaining_parts = analyse(remaining_parts, store_parts)


store: >Brick Takeover </a>
Found 15 out of 47 parts, around 31.91 %
These parts go by 62.52 euros!
store: >Brick24 - MEGASTORE 🏬</a>
Found 4 out of 47 parts, around 8.51 %
These parts go by 3.74 euros!
store: >brickina - up to 75%OFF</a>
Found 8 out of 47 parts, around 17.02 %
These parts go by 14.30 euros!
store: >Bunte Steinewelt</a>
Found 3 out of 47 parts, around 6.38 %
These parts go by 0.78 euros!
store: >German Brick Circus BULK </a>
Found 2 out of 47 parts, around 4.26 %
These parts go by 220.00 euros!
store: >KiSebA</a>
Found 16 out of 47 parts, around 34.04 %
These parts go by 22.41 euros!
store: >Magic Magnus 🛒 %SALE%</a>
Found 0 out of 47 parts, around 0.00 %
These parts go by 0.00 euros!
store: >Sky Bricks shop !! -40% !!</a>
Found 7 out of 47 parts, around 14.89 %
These parts go by 169.00 euros!
store: >Sta Laedla</a>
Found 24 out of 47 parts, around 51.06 %
These parts go by 124.40 euros!
store: >The.Brick.Factory</a>
Found 7 out of 47 parts, around 14.89 %
These parts 

In [687]:
part_set = set()

# for every dict, add the part to a set if not none

for part_dict in evr:
    for key in part_dict.keys():
        if part_dict[key] is not None:
            part_set.add(key)

In [689]:
len(part_set)

43

In [690]:
len(remaining_parts)

47

In [691]:
for part in remaining_parts:
    if part not in part_set:
        print(part)

63869pb001
24116pb004
24116pb003L
3068pb1039


In [11]:
driver = webdriver.Chrome()
driver.get(volvo_set_url)
soup = BeautifulSoup(driver.page_source,"html.parser")


In [701]:
parts = []
for line in soup.find_all("a"):
    test = str(line).split("Part")
    if len(test)>1: 
        test2 =(str(test[1]).split(" ")) 
        if len(test2) >4 and test2[1]=="No:":
                if test2[2]=="63869pb001":
                      break
                parts.append(test2[2])

qnt = []
for line in soup.find_all("td", align="RIGHT"):
        test = re.findall(r'(\d+)', str(line))  # Find digits within the td tag
        if len(test)==1:
            qnt.append(test)

my_dic = {(parts[i]):int(qnt[i][0]) for i in range(len(parts))}


In [707]:
soup.find_all("b", string="Counterparts:")

[<b>Counterparts:</b>, <b>Counterparts:</b>]

In [711]:
new_soup = []
for line in soup:
    if line.find_all("b", string="Counterparts:") is not None:
        break
    else:
        new_soup.append(line)

parts = []
for line in new_soup.find_all("a"):
    test = str(line).split("Part")
    if len(test)>1: 
        test2 =(str(test[1]).split(" ")) 
        if len(test2) >4 and test2[1]=="No:":
                if test2[2]=="63869pb001":
                      break
                parts.append(test2[2])

qnt = []
for line in soup.find_all("td", align="RIGHT"):
        test = re.findall(r'(\d+)', str(line))  # Find digits within the td tag
        if len(test)==1:
            qnt.append(test)

my_dic = {(parts[i]):int(qnt[i][0]) for i in range(len(parts))}


AttributeError: 'list' object has no attribute 'find_all'

In [720]:
lines = []
# Extract the entire text content
soup_text = soup.get_text()

# Read line by line
for line in soup_text.splitlines():
    stripped_line = line.strip()  # Remove leading and trailing whitespace
    if stripped_line:  # Skip empty lines
        lines.append(stripped_line)

In [741]:
for line in lines:
    if len (str(line).split("Summary:"))>1:
        print(line)
        break

Regular Items:Parts:*  1  42053stk01Sticker Sheet for Set 42053 - (26668/6152133) Catalog: Parts: Sticker Sheet*  1  26436Black Hose, Pneumatic 4mm D. V2 Precut 23L / 18.4cm Catalog: Parts: Hose, Pneumatic 4mm D.*  1  26440Black Hose, Pneumatic 4mm D. V2 Precut 32L / 25.6cm Catalog: Parts: Hose, Pneumatic 4mm D.*  2  26439Black Hose, Pneumatic 4mm D. V2 Precut 48L / 38.4cm Catalog: Parts: Hose, Pneumatic 4mm D.*  4  75c09Black Hose, Rigid 3mm D.  9L / 7.2cm Catalog: Parts: Hose, Rigid 3mm D.* ! 2  32199Black Hose, Soft Axle 11L Catalog: Parts: Hose, Soft Axle* ! 8  3623Black Plate 1 x 3 Catalog: Parts: Plate* ! 8  3666Black Plate 1 x 6 Catalog: Parts: Plate* ! 5  3021Black Plate 2 x 3 Catalog: Parts: Plate* ! 3  2817Black Plate, Modified 2 x 2 with Pin Holes Catalog: Parts: Plate, Modified* ! 4  4032Black Plate, Round 2 x 2 with Axle Hole Catalog: Parts: Plate, Round* ! 1  4286Black Slope 33 3 x 1 Catalog: Parts: Slope* ! 6  85970Black Slope, Curved 10 x 1 Catalog: Parts: Slope, Curved

In [746]:
str(line).split("Summary:")[1]

'\xa0Type\xa0\xa0Unique Lots\xa0\xa0Total Qty\xa0Regular Items:\xa0Parts\xa0\xa0159\xa0\xa01,166\xa0Extra Items:\xa0Parts\xa0\xa013\xa0\xa014\xa0Counterparts:\xa0Parts\xa0\xa023\xa0\xa029\xa0Source:Sealed Set Contentsfabsie007\xa0(2041) inventoried this Set on Sep 1, 2016, inventory last updated on Apr 29, 2024.Links Related to this Inventory:View a printer-friendly version of this page [with temporary checkboxes]Show Part Color Codes (?)Confirm Accuracy of this inventory if you are a registered Inventories Verifier (?)Upload an Image in the specific color if you have one and the one in the above listing is not available.Fill out an Inventory Change Request (?) [ Show Inv Item IDs ] if you found an error in this inventory.View a log of completed changes made to this inventory.See explanation on what Regular, Extra, Counterpart and Alternate means.Disclaimer: BrickLink does not guarantee an inventory to be correct and therefore an inventory should be only used as a guide.'

In [10]:
soup

NameError: name 'soup' is not defined

In [749]:
#Locate the table and extract rows
table = soup.find('table', {'class': 'ta'})  # Adjust class or ID if needed
rows = table.find_all('tr')

# Extract numbers from specific rows
data = []
for row in rows:
    cells = row.find_all('td', align='RIGHT')  # Look for specific alignment
    for cell in cells:
        text = cell.get_text(strip=True)
        if text.isdigit() or text.replace(',', '').isdigit():  # Check for numeric content
            data.append(int(text.replace(',', '')))  # Convert to int after removing commas

print(data)

[1, 1, 1, 2, 4, 2, 8, 8, 5, 3, 4, 1, 6, 2, 2, 1, 30, 13, 2, 1, 1, 9, 24, 11, 6, 16, 4, 8, 14, 2, 3, 2, 2, 1, 1, 15, 20, 2, 24, 1, 7, 14, 8, 1, 5, 31, 4, 6, 5, 5, 1, 1, 2, 2, 26, 2, 1, 2, 31, 1, 132, 7, 4, 3, 4, 4, 3, 4, 1, 1, 51, 2, 13, 96, 3, 3, 21, 5, 6, 3, 3, 1, 8, 24, 4, 3, 12, 10, 4, 7, 4, 1, 1, 1, 1, 1, 1, 1, 3, 7, 18, 1, 22, 8, 6, 1, 2, 7, 2, 3, 7, 1, 2, 8, 4, 2, 1, 3, 4, 2, 7, 3, 2, 9, 25, 5, 1, 4, 3, 5, 7, 7, 2, 3, 1, 2, 2, 1, 2, 10, 8, 4, 8, 2, 2, 1, 1, 2, 11, 2, 3, 4, 6, 9, 4, 5, 3, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 4]


In [12]:
# Locate the table
table = soup.find('table', {'class': 'ta'})  # Adjust class or ID if needed

# Extract rows from the table
rows = table.find_all('tr')

# Extract numbers only from the center columns
center_column_data = []
for row in rows:
    cells = row.find_all('td')  # Get all columns in the row
    print(cells)

[<td align="CENTER" width="10%"><b> Image </b></td>, <td align="CENTER" width="5%"><b> Qty </b></td>, <td width="10%"><b> Item No </b></td>, <td width="75%"><b>Description </b></td>, <td align="CENTER" width="5%"><b> MID </b></td>]
[<td align="CENTER" colspan="5"><font color="#FFFFFF" face="Tahoma,Arial" size="2"><b>Regular Items:</b></font></td>]
[<td align="CENTER" colspan="5"><font face="Tahoma,Arial" size="2"><b>Parts:</b></font></td>]
[<td align="CENTER"><b><a href="/catalogItemPic.asp?P=42053stk01" id="imgLink0" rel="blcatimg"><img alt="Part No: 42053stk01  Name: Sticker Sheet for Set 42053 - (26668/6152133)" border="0" height="60" id="img0" name="img0" onerror="killImage('img0');" src="https://img.bricklink.com/P/0/42053stk01.jpg" style="display:block;" title="Part No: 42053stk01  Name: Sticker Sheet for Set 42053 - (26668/6152133)" width="80"/></a><br/><font face="Tahoma,Arial" size="1">*</font> <font face="Tahoma,Arial" id="d3d42053stk01" size="1"></font><script>if (brickList[

In [38]:
# find part type tag

for row in rows:
    print(len(str(row).split("Extra")))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [59]:
regular_items=[]
extra_items=[]
counterpart_items = []
part_type = 0

for row in rows:
    if len(str(row).split("Regular"))>1:
        part_type = "regular"
    elif len(str(row).split("Extra"))>1:
        part_type = "extra"
    elif len(str(row).split("Counterpart"))>1:
        part_type = "counterpart"
    
    if part_type=="regular":
        regular_items.append(row)
    elif part_type == "extra":
        extra_items.append(row)
    elif part_type=="counterpart":
        counterpart_items.append(row)
    else:
        continue


    

In [65]:
for line in counterpart_items:

    section = str(line).split("Part No:")
    if len(section)>1:
        number = section[1].split()[0]
        print(number)
        break


63869pb001


In [72]:
section

['<tr bgcolor="#FFFFFF" class="IV_63869pb001 IV_ITEM"><td align="CENTER"><b><a href="/catalogItemPic.asp?P=63869pb001" id="imgLink172" rel="blcatimg"><img alt="',
 ' 63869pb001  Name: Technic, Axle and Pin Connector Perpendicular Triple with Orange Direction Indicator and Headlight Pattern (Sticker) - Set 42053" border="0" height="60" id="img172" name="img172" onerror="killImage(\'img172\');" src="https://img.bricklink.com/P/11/63869pb001.jpg" style="display:block;" title="',
 ' 63869pb001  Name: Technic, Axle and Pin Connector Perpendicular Triple with Orange Direction Indicator and Headlight Pattern (Sticker) - Set 42053" width="80"/></a><br/><font face="Tahoma,Arial" size="1">*</font>\xa0<font face="Tahoma,Arial" id="d3d63869pb001" size="1"></font><script>if (brickList["63869pb001"]) $(\'#d3d63869pb001\').text(\'!\')</script></b></td><td align="RIGHT">\xa01\xa0</td><td nowrap="">\xa0<a href="/v2/catalog/catalogitem.page?P=63869pb001&amp;idColor=11">63869pb001</a></td><td><b>Black Te

In [11]:
import pandas as pd
def new_get_set_parts(url):

        
    # Initialize the WebDriver with the options
    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)
    soup = BeautifulSoup(driver.page_source,"html.parser")


    # find qnt of each type of parts
    summary=soup.find_all("td", align="RIGHT")
    reg_num=int(str(summary[-6]).split("\xa0")[1])
    extra_num=int(str(summary[-4]).split("\xa0")[1])
    counter_num=int(str(summary[-2]).split("\xa0")[1])

    parts = []
    for line in soup.find_all("a"):
        test = str(line).split("Part")
        if len(test)>1: 
            test2 =(str(test[1]).split(" ")) 
            if len(test2) >4 and test2[1]=="No:":
                    parts.append(test2[2])

    qnt = []
    for line in soup.find_all("td", align="RIGHT"):
            test = re.findall(r'(\d+)', str(line))  # Find digits within the td tag
            if len(test)==1:
                qnt.append(int(test[0]))


    df = pd.DataFrame({"part number":parts,
                    "part quantity":qnt[:len(parts)],
                    "part type":["reg" for i in range(reg_num)]+["extra" for i in range(extra_num)]+["counter" for i in range(counter_num)]
                    })
    return df

In [204]:
volvo_parts = new_get_set_parts("https://www.bricklink.com/catalogItemInv.asp?S=42102-1")
volvo_parts

,part number,part quantity,part type
0,42102stk01,1,reg
1,3705,1,reg
2,32013,4,reg
3,42003,3,reg
4,32270,3,reg
5,32498,2,reg
6,43857,4,reg
7,32140,3,reg
8,33299,1,reg
9,2780,13,reg


In [205]:


driver = webdriver.Chrome()
driver.get(magic_magnus_store_url)
driver.implicitly_wait(5)

part_price = []


for id_ in range(volvo_parts.shape[0]):
    part_num = volvo_parts.iloc[id_]["part number"]
    part_qnt = int(volvo_parts.iloc[id_]["part quantity"])
    part_type = volvo_parts.iloc[id_]["part type"]

    if search_for_part(part_num,driver):
        store_part_dict = get_store_parts(driver)

        part_price.append(get_part_price(part_num, part_qnt, store_part_dict))
    else:
        part_price.append(None)

In [224]:
store_df = pd.DataFrame({
    "part number":volvo_parts["part number"],
    "part price":part_price,
    "part type":volvo_parts["part type"]
})

store_df

,part number,part price,part type
0,42102stk01,NaN,reg
1,3705,0.0200,reg
2,32013,0.0864,reg
3,42003,0.0648,reg
4,32270,0.1320,reg
5,32498,NaN,reg
6,43857,0.2880,reg
7,32140,0.0480,reg
8,33299,0.0550,reg
9,2780,0.1456,reg


In [169]:
part_num = volvo_parts.iloc[id_]["part number"]
part_qnt = volvo_parts.iloc[id_]["part quantity"]
part_type = volvo_parts.iloc[id_]["part type"]

In [220]:
volvo_parts

,part number,part quantity,part type
0,42102stk01,1,reg
1,3705,1,reg
2,32013,4,reg
3,42003,3,reg
4,32270,3,reg
5,32498,2,reg
6,43857,4,reg
7,32140,3,reg
8,33299,1,reg
9,2780,13,reg


In [215]:
len(part_price)

54

# New

In [12]:
set_link = "https://www.bricklink.com/catalogItemInv.asp?S=42053-1"

store_link = magic_magnus_store_url
set_parts = new_get_set_parts(set_link)

In [46]:
def new_get_store_parts(store_url, set_parts ):



    driver = webdriver.Chrome(options=chrome_options)
    driver.get(store_url)
    driver.implicitly_wait(5)

    part_price = []

    total = set_parts.shape[0]


    for id_ in range(set_parts.shape[0]):
        #print(f" \rProgress: {100*id_/total:.2f} %", end="") 
        part_num = set_parts.iloc[id_]["part number"]
        part_qnt = int(set_parts.iloc[id_]["part quantity"])

        if search_for_part(part_num,driver):
            store_part_dict = get_store_parts(driver)

            part_price.append(get_part_price(part_num, part_qnt, store_part_dict))
        else:
            part_price.append(None)

    store_df = pd.DataFrame({
        "part number":set_parts["part number"],
        "part price":part_price,
        "part type":set_parts["part type"],
        "part quantity":set_parts["part quantity"]
    })

    return store_df


In [ ]:
store_df = new_get_store_parts(store_link, set_parts)

Progress: 99.49 %

In [ ]:
store_df

,part number,part price,part type,part quantity
0,42053stk01,NaN,reg,1
1,26436,NaN,reg,1
2,26440,NaN,reg,1
3,26439,NaN,reg,2
4,75c09,NaN,reg,4
...,...,...,...,...
190,6636pb141R,NaN,counter,1
191,6636pb140,NaN,counter,2
192,3068pb1039,NaN,counter,3
193,18939c06,NaN,counter,1


In [ ]:
def new_analyse(store_df):
    reg = store_df[store_df["part type"] == "reg"]
    extra = store_df[store_df["part type"] == "extra"]
    counter = store_df[store_df["part type"] == "counter"]

    reg_per = 100-100*(reg["part price"].isna().sum())/reg["part price"].shape[0]
    extra_per = 100-100*(extra["part price"].isna().sum())/extra["part price"].shape[0]
    counter_per = 100-100*(counter["part price"].isna().sum())/counter["part price"].shape[0]

    print(f"Regular found: {reg_per:.2f}%")
    print(f"Extra found: {extra_per:.2f}%")
    print(f"Counter found: {counter_per:.2f}%\n")

    return store_df[store_df["part price"].isna()]



In [ ]:
remaining_from_magic = new_analyse(store_df)

Regular found: 83.02%
Extra found: 92.31%
Counter found: 4.35%


In [ ]:

store_pre_http = "https://store.bricklink.com/"
all_stores_dict = get_stores(stores_url)



In [ ]:
results = []
for id_,store in enumerate(all_stores_dict.keys()):
    store_df_ = new_get_store_parts(store_pre_http+all_stores_dict[store], remaining_from_magic)
    dummy=new_analyse(store_df_)
    results.append(store_df_)
    print(f"\r Over all Progress: {100*id_/len(all_stores_dict.keys()):.2f} %", end="") 


Regular found: 59.26%
Extra found: 100.00%
Counter found: 0.00%

 Over all Progress: 0.00 %Regular found: 14.81%
Extra found: 0.00%
Counter found: 0.00%

 Over all Progress: 2.27 %Regular found: 29.63%
Extra found: 100.00%
Counter found: 0.00%

 Over all Progress: 4.55 %Regular found: 14.81%
Extra found: 100.00%
Counter found: 0.00%

 Over all Progress: 6.82 %Regular found: 11.11%
Extra found: 0.00%
Counter found: 0.00%

 Over all Progress: 9.09 %Regular found: 55.56%
Extra found: 100.00%
Counter found: 4.55%

 Over all Progress: 11.36 %Regular found: 0.00%
Extra found: 0.00%
Counter found: 0.00%

 Over all Progress: 13.64 %Regular found: 22.22%
Extra found: 100.00%
Counter found: 0.00%

 Over all Progress: 15.91 %Regular found: 96.30%
Extra found: 100.00%
Counter found: 0.00%

 Over all Progress: 18.18 %Regular found: 25.93%
Extra found: 100.00%
Counter found: 0.00%

 Over all Progress: 20.45 %Regular found: 22.22%
Extra found: 100.00%
Counter found: 0.00%

 Over all Progress: 22.73 %

In [ ]:
for df in results:
    new_analyse(df)

Regular found: 55.56%
Extra found: 100.00%
Counter found: 0.00%

Regular found: 14.81%
Extra found: 0.00%
Counter found: 0.00%

Regular found: 29.63%
Extra found: 100.00%
Counter found: 0.00%

Regular found: 11.11%
Extra found: 0.00%
Counter found: 0.00%

Regular found: 7.41%
Extra found: 0.00%
Counter found: 0.00%

Regular found: 55.56%
Extra found: 100.00%
Counter found: 4.55%

Regular found: 0.00%
Extra found: 0.00%
Counter found: 0.00%

Regular found: 22.22%
Extra found: 100.00%
Counter found: 0.00%

Regular found: 96.30%
Extra found: 100.00%
Counter found: 0.00%

Regular found: 25.93%
Extra found: 100.00%
Counter found: 0.00%

Regular found: 22.22%
Extra found: 100.00%
Counter found: 0.00%

Regular found: 25.93%
Extra found: 100.00%
Counter found: 0.00%

Regular found: 14.81%
Extra found: 0.00%
Counter found: 0.00%

Regular found: 96.30%
Extra found: 100.00%
Counter found: 4.55%

Regular found: 40.74%
Extra found: 100.00%
Counter found: 0.00%

Regular found: 62.96%
Extra found: 10

In [ ]:
def check_missing_parts(df_list):
    parts_found = set()

    for df in df_list:
        df_ = df[["part number", "part price"]]
        for i in range(df_.shape[0]):
            if not df_.iloc[i]["part price"] :
                parts_found.add(df_.iloc[i]["part number"])
    parts_to_find = set(df_["part number"].to_list())

    missing_parts = [part for part in parts_to_find if part not in parts_found ]
    print(missing_parts)

In [ ]:
check_missing_parts(results) # no missing parts!

[]


# choose how to buy

Remove counter parts

In [383]:
filtered_results = []
for df in results:
    df_ = df[df["part type"] !="counter"]
    filtered_results.append(df_.drop("part type", axis=1))

In [387]:
len(results)

44

In [432]:
prices = []
percentage = []
for df in results:
    df_ =df.dropna()
    df_["price"]= df_["part price"]*df_["part quantity"].astype(int)
    prices.append(df_.price.sum())
    percentage.append(100*df_.shape[0]/df.shape[0])

C:\Users\diogo\AppData\Local\Temp\ipykernel_19092\405293515.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["price"]= df_["part price"]*df_["part quantity"].astype(int)
C:\Users\diogo\AppData\Local\Temp\ipykernel_19092\405293515.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["price"]= df_["part price"]*df_["part quantity"].astype(int)
C:\Users\diogo\AppData\Local\Temp\ipykernel_19092\405293515.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [469]:

final_df = pd.DataFrame({
    "percentage":percentage,
    "price":prices
})

final_df

,percentage,price
0,34.0,133.8000
1,8.0,43.9250
2,18.0,24.7705
3,10.0,10.5322
4,6.0,375820.0000
5,34.0,38.6365
6,0.0,0.0000
7,14.0,462.9720
8,54.0,283.4450
9,16.0,14.8260


In [446]:
final_df.sort_values(by="percentage",ascending=0)

,store,percentage,price
32,>GetStones24 *AUTUMN-SALE*</a>,68.0,110.8320
13,>3 BRICKS sale -15%</a>,56.0,197.8985
8,>Sta Laedla</a>,54.0,283.4450
27,>BS-Bricks</a>,50.0,681.6000
16,>Alpine Bricks</a>,40.0,13184.5400
18,>AW's Brick Collection</a>,38.0,99.3400
15,>Almost all at Sajmon´s</a>,36.0,6356.4980
0,>Brick Takeover </a>,34.0,133.8000
5,>KiSebA</a>,34.0,38.6365
28,>BSL BRICKS - USED PARTS</a>,32.0,54.4250


In [453]:
list(all_stores_dict.keys())[32]

'>GetStones24 *AUTUMN-SALE*</a>'

In [463]:
a=results[32].dropna()
a

,part number,part price,part type,part quantity
5,32199,1.300,reg,2
13,18943,0.580,reg,2
15,18938,1.170,reg,1
18,3737,0.200,reg,2
59,6628,1.336,reg,1
68,26288c01,3.384,reg,1
69,19482c01,12.810,reg,1
75,4694cc01,7.920,reg,3
124,32062,0.500,reg,25
126,3708,0.180,reg,1


In [462]:
b=a["part price"] * a["part quantity"].astype(int)
b.sum()

110.832

In [468]:

new_analyse(results[8])

Regular found: 96.30%
Extra found: 100.00%
Counter found: 0.00%



,part number,part price,part type,part quantity
15,18938,NaN,reg,1
172,63869pb001,NaN,counter,1
173,32316pb021,NaN,counter,1
174,32316pb022,NaN,counter,1
175,40490pb033,NaN,counter,1
176,40490pb031L,NaN,counter,1
177,40490pb031R,NaN,counter,1
178,40490pb034,NaN,counter,1
179,40490pb032,NaN,counter,1
180,32278pb019,NaN,counter,1


# new attempt!!

#### get the stores

In [192]:
stores_url = "https://www.bricklink.com/browse.asp"
stores_dict = get_stores(stores_url) # store name: store extentions

#### get the set

In [193]:
set_url = "https://www.bricklink.com/catalogItemInv.asp?S=42043-1"
set_parts_df = new_get_set_parts(set_url)

#### Process the set parts

remove duplicates

remove coutnter option

In [194]:
def process_set(set_parts_df, counter=False):
    #remove duplicates
    set_no_duplicate = set_parts_df.groupby("part number", as_index=False).agg({
    "part quantity": "sum",
    "part type": "first"  # Keep the first part type for each group
})
    
    if counter == False:
        set_no_duplicate = set_no_duplicate[set_no_duplicate["part type"]!= "counter"]

    return set_no_duplicate

In [195]:
clean_set_parts_df = process_set(set_parts_df)

#### sweep the stores

In [196]:
store_pre_http = "https://store.bricklink.com/"

df_per_store_list = []

for idx, store in enumerate(stores_dict.keys()):
    store_df = new_get_store_parts(store_pre_http+stores_dict[store],clean_set_parts_df)
    store_df["store"] = store
    df_per_store_list.append(store_df)
    print(f"\r Over all Progress: {100*idx/len(stores_dict.keys()):.2f} %", end="") 


 Over all Progress: 80.00 %

#### sweep the df stores and get the most of the parts in the necessary stores

In [200]:
import numpy as np

def get_df_part_percentage(df):
    return 100-(df["part price"].isna().sum()/df.shape[0])*100

def get_all_parts(stores_df_list):
    stores_df_list=stores_df_list.copy()
    found_stores=[]
    #get the percentages
    percentages = [get_df_part_percentage(df) for df in stores_df_list]

    max_idx = np.argmax(percentages)

    most_complete_df = stores_df_list[max_idx]

    # is there any remaining part?
    remaining_parts = most_complete_df[most_complete_df["part price"].isna()]
    found_stores.append(most_complete_df[most_complete_df["part price"].notna()])

    while (remaining_parts.shape[0]>0):
        print("here!")
        
        #remove most_complete_df from list
        del stores_df_list[max_idx]

        #update the remaining dfs
        for idx, df in enumerate(stores_df_list):
            stores_df_list[idx] = df[most_complete_df["part price"].isna()]
        
        # repeat the above!

        #get the percentages
        percentages = [get_df_part_percentage(df) for df in stores_df_list]
        if len (percentages)==0:
            break

        max_idx = np.argmax(percentages)

        most_complete_df = stores_df_list[max_idx]

        # is there any remaining part?
        remaining_parts = most_complete_df[most_complete_df["part price"].isna()]
        found_stores.append(most_complete_df[most_complete_df["part price"].notna()])
    
    return found_stores

found_parts = get_all_parts(df_per_store_list)

here!
here!
here!
here!
here!


# get the price of the found set

In [201]:
def get_df_price(df):
    return sum(df["part quantity"]*df["part price"])

def get_set_price(found_parts):
    return sum([get_df_price(df) for df in found_parts])

get_set_price(found_parts)

13068.616500000004

# rubish

In [219]:
df = found_parts[0]

In [224]:
# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

# All dataframes hereafter reflect these changes.
display(df)

,part number,part price,part type,part quantity,store
0,10197,0.1870,reg,2,>3 BRICKS sale -15%</a>
1,10928,0.2720,reg,4,>3 BRICKS sale -15%</a>
2,11214,1.7340,reg,34,>3 BRICKS sale -15%</a>
3,11478,0.2550,reg,2,>3 BRICKS sale -15%</a>
4,11946,0.4590,reg,3,>3 BRICKS sale -15%</a>
5,11947,0.4590,reg,3,>3 BRICKS sale -15%</a>
6,14720,0.9690,reg,3,>3 BRICKS sale -15%</a>
7,15100,1.9380,reg,19,>3 BRICKS sale -15%</a>
8,15458,1.4025,reg,3,>3 BRICKS sale -15%</a>
11,15459,12.3250,reg,10,>3 BRICKS sale -15%</a>
